# 获取数据

In [71]:
import pandas_ta as ta
import numpy as np
import pandas as pd

In [72]:
daily_300 = pd.read_csv("000300.csv", header=0, names=[ "trade_date",  "high", "low","close", "pct_chg"]).sort_values('trade_date').reset_index()
daily_300

,index,trade_date,high,low,close,pct_chg
0,4422,20050104,994.77,980.66,982.79,-1.72
1,4421,20050105,997.32,979.88,992.56,0.99
2,4420,20050106,993.79,980.33,983.17,-0.95
3,4419,20050107,995.71,979.81,983.96,0.08
4,4418,20050110,993.96,979.79,993.88,1.01
...,...,...,...,...,...,...
4418,4,20230313,4008.69,3962.64,4008.69,1.05
4419,3,20230314,4001.94,3946.94,3984.70,-0.60
4420,2,20230315,4018.75,3986.90,3986.90,0.06
4421,1,20230316,3981.32,3936.48,3939.15,-1.20


# 计算指标

In [73]:
import pandas_ta as ta
import numpy as np
import pandas as pd
pd.DataFrame().ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [74]:
def calc_CMO(mkt_data, n=14, method='TA'):
    close = mkt_data['close']
    """ 计算指标 """
    if method=='TA':
        # 1. PANDAS-TA默认参数-->按照TA，走rma的逻辑
        CMO = ta.cmo(close, length=n, talib=True)
    elif method=='own':
        # 2. 自行计算， 同 CMO = ta.cmo(close, length=n, talib=False)
        up = close.rolling(2).max()-close.shift(1)
        down = close.shift(1) - close.rolling(2).min()
        up_n = up.rolling(n).sum()
        down_n = down.rolling(n).sum()
        CMO = (up_n - down_n)/(up_n + down_n) * 100
    """ 指标赋值 """
    if method=='own':
        mkt_data['up'] = up
        mkt_data['down'] = down
        mkt_data['up_n'] = up_n
        mkt_data['down_n'] = down_n
    mkt_data['CMO'] = CMO
    return mkt_data

# 计算信号

In [75]:
def calc_signal(mkt_data):
    CMO = mkt_data['CMO'] 
    """ 计算信号 """
    signals = []
    for cmo, cmo1 in zip(CMO, CMO):
        signal = None
        if cmo>0:
            signal = 1
        elif cmo<0:
            signal = -1
        signals.append(signal)
    """ 信号赋值 """
    mkt_data['signal'] = signals
    return mkt_data

def calc_signal(mkt_data, m=0):
    CMO = mkt_data['CMO'] 
    """ 计算信号 """
    signals = []
    for cmo, cmo1 in zip(CMO, CMO):
        signal = None
        if cmo>m:
            signal = 1
        elif cmo<-m:
            signal = -1
        else:
            signal = 0
        signals.append(signal)
    """ 信号赋值 """
    mkt_data['signal'] = signals    
    return mkt_data

# 计算持仓

In [76]:
def calc_position(mkt_data):
    mkt_data['position'] = mkt_data['signal'].fillna(method='ffill').shift(1).fillna(0)
    return mkt_data

# 计算结果

In [77]:
def statistic_performance(mkt_data, r0=0.03, data_period=1440):
    position = mkt_data['position']
    
    """      序列型特征 
        hold_r :      持仓收益
        hold_win :    持仓胜负
        hold_cumu_r : 累计持仓收益
        drawdown :    回撤
        ex_hold_r :   超额收益
    """
    hold_r = mkt_data['pct_chg']/100 * position
    hold_win = hold_r>0
    hold_cumu_r = (1+hold_r).cumprod() - 1
    drawdown = (hold_cumu_r.cummax()-hold_cumu_r)/(1+hold_cumu_r).cummax()    
    ex_hold_r= hold_r-r0/(250*1440/data_period)

    mkt_data['hold_r'] = hold_r
    mkt_data['hold_win'] = hold_win
    mkt_data['hold_cumu_r'] = hold_cumu_r
    mkt_data['drawdown'] = drawdown
    mkt_data['ex_hold_r'] = ex_hold_r
    
    """       数值型特征 
        v_hold_cumu_r：         累计持仓收益
        v_pos_hold_times：      多仓开仓次数
        v_pos_hold_win_times：  多仓开仓盈利次数
        v_pos_hold_period：     多仓持有周期数
        v_pos_hold_win_period： 多仓持有盈利周期数
        v_neg_hold_times：      空仓开仓次数
        v_neg_hold_win_times：  空仓开仓盈利次数
        v_neg_hold_period：     空仓持有盈利周期数
        v_neg_hold_win_period： 空仓开仓次数
        v_hold_period：         持仓周期数（最后一笔未平仓订单也算）
        v_hold_win_period：     持仓盈利周期数（最后一笔未平仓订单也算）
        v_max_dd：              最大回撤
        v_annual_std：          年化标准差
        v_annual_ret：          年化收益
        v_sharpe：              夏普率
    """
    v_hold_cumu_r = hold_cumu_r.tolist()[-1]

    v_pos_hold_times= 0 
    v_pos_hold_win_times = 0
    v_pos_hold_period = 0
    v_pos_hold_win_period = 0
    v_neg_hold_times= 0 
    v_neg_hold_win_times = 0
    v_neg_hold_period = 0
    v_neg_hold_win_period = 0
    for w, r, pre_pos, pos in zip(hold_win, hold_r, position.shift(1), position):
        # 有换仓（先结算上一次持仓，再初始化本次持仓）
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是循环的第一次，此时无需结算，直接初始化持仓即可
            if pre_pos == pre_pos:
                # 结算上一次持仓
                if pre_pos>0:
                    v_pos_hold_times += 1
                    v_pos_hold_period += tmp_hold_period
                    v_pos_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:
                        v_pos_hold_win_times+=1
                elif pre_pos<0:
                    v_neg_hold_times += 1      
                    v_neg_hold_period += tmp_hold_period
                    v_neg_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:                    
                        v_neg_hold_win_times+=1
            # 初始化本次持仓
            tmp_hold_r = r
            tmp_hold_period = 0
            tmp_hold_win_period = 0
        else: # 未换仓
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_hold_win_period += 1
                if abs(r)>0:
                    tmp_hold_r = (1+tmp_hold_r)*(1+r)-1       

    v_hold_period = (abs(position)>0).sum()
    v_hold_win_period = (hold_r>0).sum()
    v_max_dd = drawdown.max()    
    v_annual_ret = pow( 1+v_hold_cumu_r, 
                      1/(data_period/1440*len(mkt_data)/250) )-1
    v_annual_std = ex_hold_r.std() * np.sqrt(250*1440/data_period) 
    v_sharpe= v_annual_ret / v_annual_std

    """ 生成Performance DataFrame """
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓胜率', '空仓平均持有期', 
                        '日胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益', '年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(v_hold_cumu_r),
                          v_pos_hold_times, '{:.2%}'.format(v_pos_hold_win_times/v_pos_hold_times), 
                                            '{:.2f}'.format(v_pos_hold_period/v_pos_hold_times),
                          v_neg_hold_times, '{:.2%}'.format(v_neg_hold_win_times/v_neg_hold_times), 
                                            '{:.2f}'.format(v_neg_hold_period/v_neg_hold_times),
                          '{:.2%}'.format(v_hold_win_period/v_hold_period), 
                          '{:.2%}'.format(v_max_dd), 
                          '{:.2f}'.format(v_annual_ret/v_max_dd),
                          '{:.2%}'.format(v_annual_ret), 
                          '{:.2%}'.format(v_annual_std), 
                          '{:.2f}'.format(v_sharpe)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    return mkt_data, performance_df

# 可视化

In [78]:
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [79]:
def visualize_performance(mkt_data):
    mkt_data['trade_datetime'] = mkt_data['trade_date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
    dt = mkt_data['trade_datetime']

    f1 = figure(height=300, width=700, 
                sizing_mode='stretch_width', 
                title='Target Trend',
                x_axis_type='datetime',
                x_axis_label="trade_datetime", y_axis_label="close")
    f2 = figure(height=200, sizing_mode='stretch_width', 
                title='Position',
                x_axis_label="trade_datetime", y_axis_label="position",
                x_axis_type='datetime',
                x_range=f1.x_range)
    f3 = figure(height=200, sizing_mode='stretch_width', 
                title='Return',
                x_axis_type='datetime',
                x_range=f1.x_range)
    f4 = figure(height=200, sizing_mode='stretch_width', 
                title='Drawdown',
                x_axis_type='datetime',
                x_range=f1.x_range)

    # 绘制行情
    close = mkt_data['close']
    cumu_hold_close = (mkt_data['hold_cumu_r']+1)
    f1.line(dt, close/close.tolist()[0], line_width=1)
    f1.line(dt, cumu_hold_close, line_width=1, color='red')

    # 绘制指标
#     indi = figure(height=200, sizing_mode='stretch_width', 
#                   title='KDJ',
#                   x_axis_type='datetime',
#                   x_range=f1.x_range
#                  )
    
    # 绘制仓位
    position = mkt_data['position']
    f2.step(dt, position)

    # 绘制收益
    hold_r = mkt_data['hold_r']
    f3.vbar(x=dt, top=hold_r)

    # 绘制回撤
    drawdown = mkt_data['drawdown']
    f4.line(dt, -drawdown, line_width=1)

    #p = column(f1,f2,f3,f4)
    p = gridplot([ [f1],
                   #[indi],
                   [f2], 
                   [f3],
                   [f4]
                 ])
    show(p)

## A:复现研报原策略（2005-09-01至2012-09-01） N=12

In [80]:
daily_300 = calc_CMO(daily_300, n=12, method='own')
daily_300 = calc_signal(daily_300)
daily_300 = calc_position(daily_300)
daily_300

,index,trade_date,high,low,close,pct_chg,up,down,up_n,down_n,CMO,signal,position
0,4422,20050104,994.77,980.66,982.79,-1.72,NaN,NaN,NaN,NaN,NaN,0,0.0
1,4421,20050105,997.32,979.88,992.56,0.99,9.77,0.00,NaN,NaN,NaN,0,0.0
2,4420,20050106,993.79,980.33,983.17,-0.95,0.00,9.39,NaN,NaN,NaN,0,0.0
3,4419,20050107,995.71,979.81,983.96,0.08,0.79,0.00,NaN,NaN,NaN,0,0.0
4,4418,20050110,993.96,979.79,993.88,1.01,9.92,0.00,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4418,4,20230313,4008.69,3962.64,4008.69,1.05,41.55,0.00,137.46,232.42,-25.673191,-1,-1.0
4419,3,20230314,4001.94,3946.94,3984.70,-0.60,0.00,23.99,137.46,213.81,-21.735417,-1,-1.0
4420,2,20230315,4018.75,3986.90,3986.90,0.06,2.20,0.00,139.66,196.60,-16.933325,-1,-1.0
4421,1,20230316,3981.32,3936.48,3939.15,-1.20,0.00,47.75,114.04,244.35,-36.359831,-1,-1.0


In [81]:
# 评价和展现
#result_daily_300, performance_df = statistic_performance(daily_300)
result_daily_300, performance_df = statistic_performance(daily_300[daily_300['trade_date'].apply(lambda x: x>=20050901 and x<=20120901)])
#result_daily_300, performance_df = statistic_performance(daily_300[daily_300['trade_date'].apply(lambda x: x>=20120315)])

result_daily_300
visualize_performance(result_daily_300)
print(performance_df)

C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt_data['hold_r'] = hold_r
C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt_data['hold_win'] = hold_win
C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

                 0
累计收益       865.50%
多仓次数            76
多仓胜率        46.05%
多仓平均持有期      11.11
空仓次数            76
空仓胜率        43.42%
空仓平均持有期       9.14
日胜率         55.84%
最大回撤        29.48%
年化收益/最大回撤     1.34
年化收益        39.50%
年化标准差       30.92%
年化夏普          1.28


## B: 策略优化（2012-09-01至今） N=13为例

In [82]:
daily_300 = calc_CMO(daily_300, 13, method='own')
daily_300 = calc_signal(daily_300, 5)
daily_300 = calc_position(daily_300)
daily_300

,index,trade_date,high,low,close,pct_chg,up,down,up_n,down_n,CMO,signal,position
0,4422,20050104,994.77,980.66,982.79,-1.72,NaN,NaN,NaN,NaN,NaN,0,0.0
1,4421,20050105,997.32,979.88,992.56,0.99,9.77,0.00,NaN,NaN,NaN,0,0.0
2,4420,20050106,993.79,980.33,983.17,-0.95,0.00,9.39,NaN,NaN,NaN,0,0.0
3,4419,20050107,995.71,979.81,983.96,0.08,0.79,0.00,NaN,NaN,NaN,0,0.0
4,4418,20050110,993.96,979.79,993.88,1.01,9.92,0.00,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4418,4,20230313,4008.69,3962.64,4008.69,1.05,41.55,0.00,137.46,235.72,-26.330457,-1,-1.0
4419,3,20230314,4001.94,3946.94,3984.70,-0.60,0.00,23.99,137.46,256.41,-30.200320,-1,-1.0
4420,2,20230315,4018.75,3986.90,3986.90,0.06,2.20,0.00,139.66,213.81,-20.977735,-1,-1.0
4421,1,20230316,3981.32,3936.48,3939.15,-1.20,0.00,47.75,139.66,244.35,-27.262311,-1,-1.0


In [83]:
# 评价和展现
#result_daily_300, performance_df = statistic_performance(daily_300)
#result_daily_300, performance_df = statistic_performance(daily_300[daily_300['trade_date'].apply(lambda x: x>='20050901' and x<='20120315')])
result_daily_300, performance_df = statistic_performance(daily_300[daily_300['trade_date'].apply(lambda x: x>=20120901)])

visualize_performance(result_daily_300)
print(performance_df)

C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt_data['hold_r'] = hold_r
C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt_data['hold_win'] = hold_win
C:\Users\Shuairui\AppData\Local\Temp\ipykernel_32824\3364070960.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

                0
累计收益       59.66%
多仓次数          153
多仓胜率       38.56%
多仓平均持有期      7.35
空仓次数          163
空仓胜率       32.52%
空仓平均持有期      5.30
日胜率        49.52%
最大回撤       34.03%
年化收益/最大回撤    0.14
年化收益        4.68%
年化标准差      21.44%
年化夏普         0.22
